# Web Scraping 101
In this notebook, we are going to go through a simple web scraping example. For this tutorial,
we will use a website which has publicly accessible data about a **10K** Marathon which 
happened in 2017 in Oregon, USA. Our target website is this [one](http://www.hubertiming.com/results/2017GPTR10K). On the website, there is data about each Marathon contestant such as how wel they did (place), their name, bib id and more. 

Our web scraping task is to extract all this data and save it as CSV file on our computer. We will use two main Python packages for this web scraping task: **request** and **BeatifulSoup**. These can be considered as the de facto tools for this task in Python. We will follow the follow hese steps:
- **Quickly inspect the website**: Since its a straight foward website (the data is in HTML table), we will spend little time here.
- **Open the website using request:** Here will check a few HTML tags (including table tag(tr)) and inspect their contents.
- **Retrieve data from HTML table**: This will involve some clean tasks utilising Python built-in String methods
- **Create a dataframe and save it to file:** 

# Python Setup
As usual, we import necessary packages/libraries. If you dont have the libraries, the first step would be to install them.

In [1]:
import pandas as pd
import requests
from bs4 import BeautifulSoup

# Open The Web URL Using Requests
We use the ```get``` method to open the URL and then use the ```text``` 
on the response to retrieve the HTML.

In [ ]:
url = "http://www.hubertiming.com/results/2017GPTR10K"
response = requests.get(url)
html = response.text
# Lets check if its really an HTML
print(html)

# Use BeatifulSoup to Parse the HTML
Getting the html of the page is just the first step. Next step is to create a Beautiful Soup object from the html. This is done by passing the html to the BeautifulSoup() function. The Beautiful Soup package is used to parse the html, that is, take the raw html text and break it into Python objects. The second argument ```lxml``` is the html parser whose details you do not need to worry about at this point.

In [3]:
# Create BeatifulSoup Object
soup = BeautifulSoup(html, 'lxml')
type(soup)

bs4.BeautifulSoup

## Extracting Information Using the Soup Object
The soup object allows you to extract interesting information about the website you're scraping such as getting the title of the page as shown below. For instance, we can get the title of the html. Also, you can get the text of the webpage and quickly print it out to check if it is what you expect.

In [5]:
# Get the title
title = soup.title
print(title)

<title>Race results for the 2017 Intel Great Place to Run \ Urban Clash Games!</title>


In [ ]:
# Print out the text
text = soup.get_text()

# Note how this is different from the HTML we printed above
print(soup.text)

## Extracting Information Using HTML Tags
Recall the following about HTML:
- HTML elements are the building blocks of HTML pages
- HTML elements are represented by tags
- HTML tags label pieces of content such as "heading", "paragraph", "table", and so on
- Browsers do not display the HTML tags, but use them to render the content of the page
For a full reference of tags, see [this HTML reference](https://www.w3schools.com/tags/)

We extract the information we need from the HTML using these tags, therefore its important to understand them.
The **soup** object has many methods but we will use the ```find_all()``` method to extract useful html tags within a webpage such as ```<a>``` tag for hyperlinks.

For our task here, the useful links are as below:
- ```<table>``` for tables
- ```<tr>``` for table rows
- ```<th>``` for table headers
- ``` <td>``` for table cells. 

**Just for fun, lets see how you can extract all the hyperlinks within the webpage in code below**

In [7]:
# Extract all hyperlinks
soup.find_all('a')

# the find all method returns a list
all_links = soup.find_all("href")

# Loop through the list to retrieve required attributes from the hyperlinks
for link in soup.find_all('a'):
    print(link.get("href"))

mailto:timing@hubertiming.com
https://www.hubertiming.com
/results/2017GPTR
/results/team/2017GPTR
/results/team/2017GPTR10K
/results/summary/2017GPTR10K
None
#tabs-1
https://www.hubertiming.com/
https://facebook.com/hubertiming/
None


In [ ]:
soup.find_all('a')

As you can see from the output above, html tags sometimes come with attributes such as class, src, etc. These attributes provide additional information about html elements. You can use a for loop and the get('"href") method to extract and print out only hyperlinks. 

# Convert Tabular Information from HTML into a Python DataFrame
Eventually what we would want to do is access a website, extract the information we need and then save it to disk or do some analysis with it. To get there, you should get all table rows in list form first and then convert that list into a dataframe.

In [9]:
# Lets get all table rows and inspect the rows
rows = soup.find_all('tr')  # A list of table rows
type(rows)
print(rows[5]) # print only first 5

<tr data-bib="814">
<td>1</td>
<td>814</td>
<td>

                    JARED WILSON

                </td>
<td>M</td>
<td>TIGARD</td>
<td>36:21</td>
<td>36:24</td>
<td></td>
</tr>


In [ ]:
type(rows[5])

In [10]:
# Lets check a single row
for row in rows:
    row_td = row.find_all('td')

print(row_td)
type(row_td)

[<td>577</td>, <td>443</td>, <td>

                    LIBBY B MITCHELL

                </td>, <td>F</td>, <td>HILLSBORO</td>, <td>1:41:18</td>, <td>1:42:10</td>, <td></td>]


bs4.element.ResultSet

The output above shows that each row is printed with html tags embedded in each row. This is not what we want. We can remove the html tags using Beautiful Soup or regular expressions (not recommended). The easiest way to remove html tags is to use Beautiful Soup, and it takes just one line of code to do this. Pass the string of interest into BeautifulSoup() and use the get_text() method to extract the text without html tags.

Lets examine further  the one table cell above

In [ ]:
# Convert table cell from bs4.element.ResultSet to String
# This step os required because Soup object acceps Strings
# When we want to get text from the HTML tags
str_cells = str(row_td)
print('Its now a string but it has ugly HTML tags')
print()
# print(str_cells )

# Now Lets get the text
clean_text = BeautifulSoup(str_cells, "lxml").get_text()
print()
print('No tags now')
print(clean_text)
# # And Lets see the result
# print(clean_text)

## Creating a Pandas DataFrame From The Website Data
Now that we know how to extract the text only from HTML tags,
we will loop through all rows, get cell contents, 
use BS to extract text without HTML tags from the cell, 
do some string cleaning and finally put that row into a list

In [ ]:
str_cells = str(cells)
str_cells

In [ ]:
cleantext = BeautifulSoup(str_cells, "lxml").get_text()
cleantext

In [ ]:
cleantext2 = cleantext[1:-1]
cleantext2

In [ ]:
split_str = cleantext2.split(',') 
split_str

In [15]:
def clean_table_row(row=None):
    # Extract cell using the cell HTML tag
    cells = row.find_all('td')
    
    # Convert to String
    str_cells = str(cells)
    
    # Use BS to extract only text and remove HTML tags
    cleantext = BeautifulSoup(str_cells, "lxml").get_text()
    # Note that we have brackets at each end, we remove them using list indexing
    cleantext2 = cleantext[1:-1]
    
    # This string: cleantext2 is just one whole string, so we split using the delimiter (comma)
    split_str = cleantext2.split(',') # returns a list
    if len(split_str) > 2:
        split_str[2] = split_str[2].strip()
    
    return split_str

In [19]:
one_row = rows[10]
print("ROW BEFORE: ")
print(one_row)
print("ROW AFTER: ")
clean_table_row(row=one_row)

ROW BEFORE: 
<tr data-bib="642">
<td>6</td>
<td>642</td>
<td>

                    JONATHON TRAN

                </td>
<td>M</td>
<td>PORTLAND</td>
<td>39:49</td>
<td>39:55</td>
<td></td>
</tr>
ROW AFTER: 


['6', ' 642', 'JONATHON TRAN', ' M', ' PORTLAND', ' 39:49', ' 39:55', ' ']

### Loop Through All Rows, Clean Them and Put in a List

In [39]:
# We will collect our cleaned rows in this list
cleaned_rows = []

for row in rows:
    # Clean up the row using our little function above
    clean_row = clean_table_row(row)
    #print(clean_row)
    # We are only interested in a full table row, 
    # so discard the rest of the rows using length
    if len(clean_row) > 5:
        cleaned_rows.append(clean_row)

# Do some more clean-up
The goal it to remove the  ```\n\n``` text strings

In [43]:
cleaner_rows = []
for i, row in enumerate(cleaned_rows):
    new_row = []
    for text in row:
        if "\n" in text  or "\r" in text:
            new_row.append(text.strip())
        else:
            new_row.append(text)
    cleaner_rows.append(new_row)

### Get Table Headers
We use the same process as above to clean up the headers

In [37]:
# Get Table headers using 'th' HTML tag
headers_with_tags = soup.find_all('th')

# Convert to string
headers_str = str(headers_with_tags)

# Extract text only and leave out HTML tags
headers_without_tags = BeautifulSoup(headers_str, "lxml").get_text()
headers_without_tags2 = headers_without_tags[1:-1]

# Split using comma delimeter and remove any trailing spaces
split_header = headers_without_tags2.split(',')
split_header2 = [i.strip() for i in split_header] 

# Lets check out the headers now
print(split_header2)

['Place', 'Bib', 'Name', 'Gender', 'City', 'Chip Time', 'Gun Time', 'Team']


In [ ]:
cleaned_rows

### Finally, Lets Create The DataFrame
We have a nested list: *cleaned_rows* where each element is a list containing a cells of a single table row. 
We also have column headers. We use these two to create a pandas DataFrame.

In [38]:
# Create Pandas DataFrame
df = pd.DataFrame(data=cleaner_rows, columns=split_header2)
df.head()

,Place,Bib,Name,Gender,City,Chip Time,Gun Time,Team
0,1,814,JARED WILSON,M,TIGARD,36:21,36:24,
1,2,573,NATHAN A SUSTERSIC,M,PORTLAND,36:42,36:45,INTEL TEAM F
2,3,687,FRANCISCO MAYA,M,PORTLAND,37:44,37:48,
3,4,623,PAUL MORROW,M,BEAVERTON,38:34,38:37,
4,5,569,DEREK G OSBORNE,M,HILLSBORO,39:21,39:24,INTEL TEAM F


### Save The DataFrame
Once we are happy with our dataframe, we can save it as a CSV file

In [ ]:
## Save AS CSV into data folder
out_file = '../../day2-python-for-data-science/data/marathoners.csv'

# The index = False option ensures we dont save the default index
df.to_csv(out_file, index=False)

# EXERCISE
In order to make the exercises easy and fast, I looked for a similar website for you to scrape. The [target](https://www.tcsnycmarathon.org/about-the-race/results/overall-men) website also contains Marathon results for men only. The task is the same as in the example, there is a table of Marathon results, please extract the data and save it into
a CSV file. A bonus task: for those who finish fast, please tabulate number of participants by country.

## Open URL

In [ ]:
# Open the website using requests, retrieve HTML and create BS object
url = YOUR CODE
r = YOUR CODE
html = YOUR CODE
bs = BeautifulSoup(html, 'lxml')

## Extract All Rows and Inspect Them

In [ ]:
# Extract all table rows using the tr HTML tag and inspect the first couple of rows
all_rows = YOUR CODE  # A list of table rows
print(all_rows[1]) # print only first 5

## Loop Through Rows, Clean Table Cells and Save Into a List

In [ ]:
# Declare list to hold all cleaned rows
cleaned_rows = [] 

for row in all_rows:
    # Extract cell using table cell HTML tag
    cells = YOUR CODE
    
    # Extract text only
    str_cells = str(cells)
    clean_text = BeautifulSoup(str_cells, "lxml").get_text()
    
    # Remove white spaces-a little convuluted but it works
    clean_text2 = " ".join(clean_text.split())

    # Remove brackts at beginning and end
    clean_text3 = clean_text2[1:-1]
    
    # Split clean_text3 using comma delimiter
    YOUR CODE
    
    # Remove white spaces again
    split_str2 = [i.strip() for i in split_str]
    
    # Add split_str2 to cleaned_rows list
    YOUR CODE

## Create A DataFrame and Inspect It

In [ ]:
# Column names: note that the first element of the list contains the column names
# Use list indexing to get the column headers
colnames = YOUR CODE

# Create Dataframe
df_men = pd.DataFrame(data=cleaned_rows[1:], columns=YOUR CODE)


In [ ]:
# Check the Dataframe
YOUR CODE

## Save the DataFrame to File

In [ ]:
# Save into the data folder in the ch1 workig directory
output_filename = YOUR CODE

# save to CSV file
YOUR CODE

## Bonus: Check Number of Men Marathoners by Country
Hint: Use pandas DataFrame ```value_counts()``` method for this.

In [ ]:
YOUR CODE